## Tarea 2: Limpieza del conjutno de datos

Dentro de este notebook, se trabajo en la depuración de los datos con el fin de facilitar los futuros análisis que se desarrollen a lo largo de este curso.

Impotación de librerias y lectura del conjunto de datos

In [7]:
import pandas as pd
import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
import seaborn as sns

loans = pd.read_csv('C:/Users/AranzaEstebanAvalos/Documents/Aprendizaje_Automatico/data.csv', low_memory=False)
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 855969 entries, 0 to 855968
Data columns (total 73 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           855969 non-null  int64  
 1   member_id                    855969 non-null  int64  
 2   loan_amnt                    855969 non-null  int64  
 3   funded_amnt                  855969 non-null  int64  
 4   funded_amnt_inv              855969 non-null  float64
 5   term                         855969 non-null  object 
 6   int_rate                     855969 non-null  float64
 7   installment                  855969 non-null  float64
 8   grade                        855969 non-null  object 
 9   sub_grade                    855969 non-null  object 
 10  emp_title                    806526 non-null  object 
 11  emp_length                   812908 non-null  object 
 12  home_ownership               855969 non-null  object 
 13 

In [2]:
loans.isnull().sum()

id                       0
member_id                0
loan_amnt                0
funded_amnt              0
funded_amnt_inv          0
                     ...  
total_rev_hi_lim     67313
inq_fi              842681
total_cu_tl         842681
inq_last_12m        842681
default_ind              0
Length: 73, dtype: int64

In [3]:
loans.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

In [8]:
# Eliminar columnas que se descartaran para el análisis
remove_col = [
    'emp_title',
    'issue_d',
    'desc',
    'zip_code',
    'addr_state',
    'earliest_cr_line',
    'last_pymnt_d',
    'last_pymnt_amnt',
    'next_pymnt_d',
    'last_credit_pull_d',
    'collections_12_mths_ex_med',
    'mths_since_last_major_derog', 
    'open_acc_6m',
    'open_il_6m', 
    'open_il_12m', 
    'open_il_24m', 
    'open_rv_12m' ,
    'open_rv_24m', 
    'max_bal_bc',
    'grade'
]

loans = loans.drop(remove_col, axis=1)

Reemplazar valores en colmunas con el fin de obtener valores númericos unicamente

In [9]:

# Omitir palabras en term para obtener valores numericos
loans['term'] = loans['term'].str.replace(' months', '')

# reemplazar '< 1 year' por 0's y omitir palabras 'years/year'
loans['emp_length'] = loans['emp_length'].str.replace('< 1 year', '0') 
loans['emp_length'] = loans['emp_length'].str.replace('+ years', '')
loans['emp_length'] = loans['emp_length'].str.replace(' years', '')
loans['emp_length'] = loans['emp_length'].str.replace(' year', '')

loans['emp_length']

0         10
1          0
2         10
3         10
4          1
          ..
855964     8
855965    10
855966     5
855967     1
855968    10
Name: emp_length, Length: 855969, dtype: object

Asignar valor numerico a las sub calificaciones

In [10]:
# Obtener valores unicos
loans['sub_grade'].unique

<bound method Series.unique of 0         B2
1         C4
2         C5
3         C1
4         B5
          ..
855964    B5
855965    B5
855966    D2
855967    E3
855968    B5
Name: sub_grade, Length: 855969, dtype: object>

In [12]:
subgrades = loans['sub_grade'].unique()

# Definir una función para ordenar las subcalificaciones
def custom_sort_key(subgrade):
    match = re.match(r'([A-Za-z]+)(\d+)', subgrade)
    letter = match.group(1)
    number = int(match.group(2))
    return letter, number

# Ordenar las subcalificaciones usando la función personalizada
sorted_subgrades = sorted(subgrades, key=custom_sort_key)

encoder = LabelEncoder()
encoder.fit(sorted_subgrades)

# Crear una columna 'sub_grade_encoded' con los nuevos valores
loans['sub_grade_encoded'] = encoder.transform(loans['sub_grade'])

# Crear un diccionario para idetntifacar lo nuevos valores asignados
dict_subgrade = dict(zip(sorted_subgrades, encoder.transform(sorted_subgrades)))
print(dict_subgrade)


{'A1': 0, 'A2': 1, 'A3': 2, 'A4': 3, 'A5': 4, 'B1': 5, 'B2': 6, 'B3': 7, 'B4': 8, 'B5': 9, 'C1': 10, 'C2': 11, 'C3': 12, 'C4': 13, 'C5': 14, 'D1': 15, 'D2': 16, 'D3': 17, 'D4': 18, 'D5': 19, 'E1': 20, 'E2': 21, 'E3': 22, 'E4': 23, 'E5': 24, 'F1': 25, 'F2': 26, 'F3': 27, 'F4': 28, 'F5': 29, 'G1': 30, 'G2': 31, 'G3': 32, 'G4': 33, 'G5': 34}


In [13]:
loans['purpose'].unique()
encoder = LabelEncoder()
encoder.fit(loans['purpose'])

# Obtener la lista de categorías únicas y sus valores numéricos
categorias = encoder.classes_
idcode = encoder.transform(categorias)

# Guardar valores correspondientes al numero en un diccionario dictionary
dict_purpose = dict(zip(categorias, idcode))
print(dict_purpose)


{'car': 0, 'credit_card': 1, 'debt_consolidation': 2, 'educational': 3, 'home_improvement': 4, 'house': 5, 'major_purchase': 6, 'medical': 7, 'moving': 8, 'other': 9, 'renewable_energy': 10, 'small_business': 11, 'vacation': 12, 'wedding': 13}


In [14]:
loans['verification_status'].unique

<bound method Series.unique of 0                Verified
1         Source Verified
2            Not Verified
3         Source Verified
4         Source Verified
               ...       
855964           Verified
855965           Verified
855966           Verified
855967    Source Verified
855968           Verified
Name: verification_status, Length: 855969, dtype: object>

In [16]:
data = {'verification_status': ['Not Verified', 'Source Verified', 'Verified']}

encoder = LabelEncoder()
encoder.fit(loans['verification_status'])

# Obtener la lista de categorías únicas y sus valores numéricos
categorias = encoder.classes_
valores_numericos = encoder.transform(categorias)

# Crear un diccionario para identificar el  número asignado
veristatus_dict = dict(zip(categorias, valores_numericos))
print(veristatus_dict)


{'Not Verified': 0, 'Source Verified': 1, 'Verified': 2}


Generar nueva columna con el periodo del prestamo

In [101]:

# Calcular la diferencia de días y giardar en una nueva columna 'history_length'
loans['credit_history_length'] = (pd.to_datetime(loans['last_credit_pull_d'], format="%d-%m-%Y") - pd.to_datetime(loans['earliest_cr_line'], format="%d-%m-%Y")).dt.days

# Intercambia los valores de 'credit_history_length' y 'default_ind'
loans['credit_history_length'], loans['default_ind'] = loans['default_ind'], loans['credit_history_length']
print(loans)


              id  member_id  loan_amnt  funded_amnt  funded_amnt_inv  \
0        1077501    1296599       5000         5000           4975.0   
1        1077430    1314167       2500         2500           2500.0   
2        1077175    1313524       2400         2400           2400.0   
3        1076863    1277178      10000        10000          10000.0   
4        1075358    1311748       3000         3000           3000.0   
...          ...        ...        ...          ...              ...   
855964  36371250   39102635      10000        10000          10000.0   
855965  36441262   39152692      24000        24000          24000.0   
855966  36271333   38982739      13000        13000          13000.0   
855967  36490806   39222577      12000        12000          12000.0   
855968  36271262   38982659      20000        20000          20000.0   

              term  int_rate  installment grade  sub_grade  ... open_rv_12m  \
0        36 months     10.65       162.87     B         